In [14]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "paultan.csv"
args.stemmer = "lemma"
args.dictionary = "automotive"

In [15]:
# -*- coding: utf-8 -*-
import numpy as np
import lda
import lda.datasets
import csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models

from stemming.porter2 import stem
from nltk.stem import *
import unicodecsv
import re
import pyLDAvis.gensim
import gensim

_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))

import enchant
d = enchant.Dict("en_US")
# Or using the /usr/share/dict/british-english word list
with open(args.dictionary + "-english") as word_file:
  english_words = set(word.strip().lower() for word in word_file)
  print(english_words)
  def is_english_word(word):
    return word.lower() in english_words

def process_tokens(tokens,stemmer):
  tokens = [i for i in tokens if not i in en_stop and not contains_digits(i) and is_english_word(i)]
  if stemmer == 'porter':
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(i) for i in tokens]
  elif stemmer == 'porter2':
    tokens = [stem(i) for i in tokens]
  elif stemmer == 'lemma':
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(i) for i in tokens]
  return tokens


if args.filename == "sample":
  X = lda.datasets.load_reuters()
  dictionary = lda.datasets.load_reuters_vocab()
  titles = lda.datasets.load_reuters_titles()
else:
  f = open(args.filename)
  reader = unicodecsv.reader(f, encoding='utf-8')
  # csv_length = sum(1 for row in reader)
  # f.seek(0) #reset reader position
  identifiers = reader.next()
  contents_idx = identifiers.index("contents")
  title_idx = identifiers.index("title")

  contents = [ row[contents_idx] for row in reader if row[contents_idx] ]

  f.seek(0)
  reader.next()
  titles = [ row[title_idx] for row in reader if row[contents_idx] ]



  texts = list()
  tokenizer = RegexpTokenizer(r'\w+')
  en_stop = get_stop_words('en')
  for idx,i in enumerate(contents):
#     if not idx % 10:
#       print "INFO: Tokenizing articles <{}> ".format(idx)
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(process_tokens(tokens, args.stemmer))
    # print idx
    # add tokens to list

  print "[DEBUG] Length of Texts : {}".format(len(texts))
  dictionary = corpora.Dictionary(texts)
  corpus = [dictionary.doc2bow(text) for text in texts]


set(['changfeng motor', 'hi-test', 'magnetic', 'alternatives', 'dynasty', 'obstruction', 'bonneville salt flats', 'query', 'loyalty', 'adjustment', 'integrity', 'gonow', 'synthetic', 'saleswoman', 'disposal', 'technique', 'all-wheel drive', 'corrosive', 'jalopy', 'jianghuai', 'safety', 'infant seats', 'gravel', 'roadway', 'protective', 'lessons', 'advantage', 'dynamic', 'activation', 'aston martin', 'faw', 'mph', 'liability', 'balancing', 'drinking', 'lubrication', 'courageous', 'condition', 'tickets', 'retrial', 'transmission', 'race', 'jiangnan', 'recall', 'quick', 'inspection', 'accumulation', 'triumph', 'revolution', 'occurrence', 'corvette', 'four door', 'horn', 'tires', 'cost', 'design', 'enhancement', 'rigorous', 'runner-up', 'wings', 'hongqi', 'wreck', 'fine', 'specification', 'overload', 'sequential', 'optional', 'roewe', 'international', 'changhe', 'appeal', 'racing', 'ignition', 'impact', 'zhongyu', 'jiangling', 'component', 'chassis', 'lubricant', 'hwanghai', 'pedals', 'mac

In [16]:
pyLDAvis.enable_notebook()
model_filename = args.filename.split('.')[0] + '.model'
try:
  ldamodel = models.LdaModel.load(model_filename)
except IOError:
  ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
  ldamodel.save(model_filename)
    
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      27.492607        1       1 -0.149993  0.024811
5      16.320078        1       2 -0.145662 -0.015495
4       8.728842        1       3 -0.110688  0.187853
8       8.218921        1       4  0.013224 -0.162135
1       8.156610        1       5  0.115635  0.081962
6       7.855694        1       6  0.092753 -0.122229
2       5.845695        1       7 -0.045105 -0.059236
0       5.807053        1       8  0.221900  0.123305
9       5.796255        1       9  0.101083 -0.047638
7       5.778246        1      10 -0.093147 -0.011198, topic_info=    Category         Freq          Term  Total  loglift  logprob
355  Default  1362.000000           bmw   1362  30.0000  30.0000
290  Default  1076.000000         honda   1076  29.0000  29.0000
463  Default   700.000000    volkswagen    700  28.0000  28.0000
373  Default   804.000000          audi    804  27.0000  27.0000
437  Default  1097.000000        toyota   1097  26.0000  26.0000
338  Default   857.000000        proton    857  25.0000  25.0000
252  Default  1053.000000         class   1053  24.0000  24.0000
206  Default   637.000000       service    637  23.0000  23.0000
486  Default   622.000000         coupe    622  22.0000  22.0000
436  Default   644.000000        nissan    644  21.0000  21.0000
351  Default  2477.000000        engine   2477  20.0000  20.0000
282  Default   898.000000          fuel    898  19.0000  19.0000
317  Default   624.000000         mazda    624  18.0000  18.0000
78   Default  1393.000000       vehicle   1393  17.0000  17.0000
177  Default   384.000000       hyundai    384  16.0000  16.0000
295  Default  5086.000000           new   5086  15.0000  15.0000
156  Default  2010.000000          year   2010  14.0000  14.0000
500  Default   591.000000          ford    591  13.0000  13.0000
27   Default  3567.000000           car   3567  12.0000  12.0000
37   Default  1174.000000        design   1174  11.0000  11.0000
173  Default   471.000000       highway    471  10.0000  10.0000
168  Default   909.000000        safety    909   9.0000   9.0000
395  Default   357.000000           kia    357   8.0000   8.0000
346  Default   342.000000       porsche    342   7.0000   7.0000
26   Default   392.000000          race    392   6.0000   6.0000
389  Default   847.000000        driver    847   5.0000   5.0000
87   Default   255.000000    mitsubishi    255   4.0000   4.0000
493  Default   270.000000        suzuki    270   3.0000   3.0000
308  Default   225.000000       perodua    225   2.0000   2.0000
46   Default   338.000000        racing    338   1.0000   1.0000
..       ...          ...           ...    ...      ...      ...
63   Topic10    11.657915        secure     38   1.9986  -5.6495
411  Topic10     5.478070     ergonomic      9   2.0734  -7.0151
418  Topic10    10.811200     catalytic     24   1.9464  -6.1613
247  Topic10    66.158135     automaker    228   1.5859  -4.2705
525  Topic10    18.082486         rally     69   1.7528  -5.2988
89   Topic10    31.865058        recall    112   1.5877  -4.9796
27   Topic10   267.791674           car   3567   0.2701  -2.8362
295  Topic10   280.144663           new   5086  -0.0975  -2.8491
55   Topic10    34.802734     technical    190   1.1745  -4.8643
175  Topic10   122.661149         model   1536   0.3175  -3.6313
373  Topic10    83.837739          audi    804   0.5735  -4.0226
467  Topic10    23.085938  manufacturer    110   1.3606  -5.2247
156  Topic10   114.519984          year   2010  -0.0747  -3.7546
351  Topic10   127.716611        engine   2477  -0.1850  -3.6559
536  Topic10    44.525861          test    469   0.4748  -4.6604
78   Topic10    72.262754       vehicle   1393  -0.1232  -4.1697
144  Topic10    43.532308         debut    532   0.3513  -4.6578
60   Topic10    24.951151        action    163   0.9278  -5.2642
359  Topic10    20.262030         value    116   1.1401  -5